In [2]:
import os
%pwd

'/home/sermengi/text-summarizer/research'

In [3]:
os.chdir("../")
!pwd

/home/sermengi/text-summarizer


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
   root_dir: Path
   source_URL: Path
   local_data_file: Path
   unzip_dir: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

In [17]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info("File is downloaded")
        else:
            logger.info("File already exists.")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)    

In [19]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-07-23 15:50:52,800: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-23 15:50:52,802: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-23 15:50:52,803: INFO: common: created directory at: artifacts]
[2025-07-23 15:50:52,804: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-23 15:50:54,142: INFO: 699133900: File is downloaded]
